# Health Insurance Claims Analysis
#### The following analysis was performed on a sample of the Health Insurance Claims Data from the state of Utah that were filed in 2016. The intention was to train and test machine learning algorithms on the data to predict the cost of a procedure based on features I engineered. Because caesarian sections are the most common procedure performed in the United States, I focused my initial analysis on that procedure. I discovered that the costs are not predictable using regression analysis, and there's a much larger story. Further digging revealed that insurance companies often negotiate directly with hospitals to set the price of what they're willing to pay for a procedure, regardless of the specifics of the patient. I'm working on expanding my project using the data pipeline I've established to look at whether my result holds true for additional procedures. 

In [21]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [22]:
caesarian_df = pd.read_csv('caesarian_data.csv', index_col=[0])

#### The short coming of using the five digit CPT4 code is that there are claims that will have that same code as part of their claim number. To eliminate those claims from the data, create a new data frame that is the claims that have those five numbers in the appropriate column as opposed to anywhere in the row.

In [23]:
caesarian_df2 = caesarian_df[caesarian_df['CPT4_ID'] == ('59510')]
caesarian_df2

,Claim_Line_No,Place_of_Service_Cd,Rev_Cd,CPT4_ID,CPT4_Mod1_Cd,CPT4_Mod2_Cd,CPT4_Mod3_Cd,CPT4_Mod4_Cd,Service_Start_Dt,Units,...,NDC_Cd,Capitation_Flag,ER_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt,Dental_Flag,Unit_Of_Measure_Cd,Unit_Of_Measure_Desc,Service_Start_Year
Claim_ID,,,,,,,,,,,,,,,,,,,,,
340993570,1,21,NaN,59510,,,,,2016-07-29,1,...,0,N,N,0.0,715.54,715.54,N,UN,Units,2016
350588632,1,21,NaN,59510,,,,,2016-06-27,1,...,0,N,N,0.0,641.99,0.00,N,UN,Units,2016
357047267,1,21,NaN,59510,UC,,,,2016-10-17,1,...,0,Y,N,0.0,0.00,0.00,N,UN,Units,2016
340423363,1,21,NaN,59510,,,,,2016-02-21,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
171159994,1,21,NaN,59510,,,,,2016-02-05,1,...,0,N,N,0.0,295.52,0.00,N,UN,Units,2016
171159994,2,21,NaN,59510,,,,,2016-02-05,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
171175684,1,21,NaN,59510,,,,,2016-01-31,1,...,0,N,N,0.0,277.24,0.00,N,UN,Units,2016
171175684,2,21,NaN,59510,,,,,2016-01-31,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016
171157745,1,99,NaN,59510,,,,,2016-02-08,1,...,0,N,N,0.0,0.00,0.00,N,UN,Units,2016


## Data Cleaning

#### It is not useful or necessary to use all of the columns for the analysis. Some are explanations of other columns, need to be turned into numeric values to be useful, or mostly populated by missing, null, or NaN values that limit their usefulness. Start by taking a look at the full list of columns and making decisions on what will be useful.

In [24]:
list(caesarian_df2.columns.values)

['Claim_Line_No',
 'Place_of_Service_Cd',
 'Rev_Cd',
 'CPT4_ID',
 'CPT4_Mod1_Cd',
 'CPT4_Mod2_Cd',
 'CPT4_Mod3_Cd',
 'CPT4_Mod4_Cd',
 'Service_Start_Dt',
 'Units',
 'Charge_Amt',
 'Prepaid_Amt',
 'Plan_Paid_Amt',
 'NDC_Cd',
 'Capitation_Flag',
 'ER_Flag',
 'Copay_Amt',
 'Coinsurance_Amt',
 'Deductible_Amt',
 'Dental_Flag',
 'Unit_Of_Measure_Cd',
 'Unit_Of_Measure_Desc',
 'Service_Start_Year']

#### Remove columns that are redundant (i.e. descriptive of another column in the table) or contain only one value

In [25]:
edited_caesarian_df = caesarian_df2.drop(['Claim_Line_No','Dental_Flag', 'Units', 'Rev_Cd', 'NDC_Cd', 'ER_Flag', 'Service_Start_Year', 'Unit_Of_Measure_Desc', 'Unit_Of_Measure_Cd'], axis=1)
edited_caesarian_df

,Place_of_Service_Cd,CPT4_ID,CPT4_Mod1_Cd,CPT4_Mod2_Cd,CPT4_Mod3_Cd,CPT4_Mod4_Cd,Service_Start_Dt,Charge_Amt,Prepaid_Amt,Plan_Paid_Amt,Capitation_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt
Claim_ID,,,,,,,,,,,,,,
340993570,21,59510,,,,,2016-07-29,3350.00,0.00,1669.58,N,0.0,715.54,715.54
350588632,21,59510,,,,,2016-06-27,3500.00,0.00,688.25,N,0.0,641.99,0.00
357047267,21,59510,UC,,,,2016-10-17,3586.00,0.00,2029.14,Y,0.0,0.00,0.00
340423363,21,59510,,,,,2016-02-21,3800.00,1932.28,96.22,N,0.0,0.00,0.00
171159994,21,59510,,,,,2016-02-05,2689.03,0.00,1182.07,N,0.0,295.52,0.00
171159994,21,59510,,,,,2016-02-05,1160.97,0.00,1160.97,N,0.0,0.00,0.00
171175684,21,59510,,,,,2016-01-31,2710.84,0.00,1108.96,N,0.0,277.24,0.00
171175684,21,59510,,,,,2016-01-31,1089.16,0.00,1089.16,N,0.0,0.00,0.00
171157745,99,59510,,,,,2016-02-08,3850.00,0.00,2638.56,N,0.0,0.00,0.00


#### The CPT4 modifications represent additional coding in the claim. While of potential use, they're unfortunately not used consistently between facilities and the columns are sparsely populated. There are very few rows (<30 out of a data frame with 6200+ rows) with CPT4 modifications. Drop them to simplify the process.

In [26]:
simplified_CPT4_df = edited_caesarian_df.drop(['CPT4_Mod1_Cd', 'CPT4_Mod2_Cd', 'CPT4_Mod3_Cd', 'CPT4_Mod4_Cd'], axis=1)
simplified_CPT4_df


,Place_of_Service_Cd,CPT4_ID,Service_Start_Dt,Charge_Amt,Prepaid_Amt,Plan_Paid_Amt,Capitation_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt
Claim_ID,,,,,,,,,,
340993570,21,59510,2016-07-29,3350.00,0.00,1669.58,N,0.0,715.54,715.54
350588632,21,59510,2016-06-27,3500.00,0.00,688.25,N,0.0,641.99,0.00
357047267,21,59510,2016-10-17,3586.00,0.00,2029.14,Y,0.0,0.00,0.00
340423363,21,59510,2016-02-21,3800.00,1932.28,96.22,N,0.0,0.00,0.00
171159994,21,59510,2016-02-05,2689.03,0.00,1182.07,N,0.0,295.52,0.00
171159994,21,59510,2016-02-05,1160.97,0.00,1160.97,N,0.0,0.00,0.00
171175684,21,59510,2016-01-31,2710.84,0.00,1108.96,N,0.0,277.24,0.00
171175684,21,59510,2016-01-31,1089.16,0.00,1089.16,N,0.0,0.00,0.00
171157745,99,59510,2016-02-08,3850.00,0.00,2638.56,N,0.0,0.00,0.00


## Adding Additional Columns from Another File


#### There are a few columns of potentially useful information in a different file. This is a massive file, so for efficiency's sake, it is easiest to focus specifically on claims that are the right procedure and match with the claims already present in the data frame. 

In [27]:
with open("Claim_Medical_Header.csv") as file:
    with open("caesarian_claims.csv", "w") as output:
        #use with to make sure it automatically gets closed
        for i,line in enumerate(file):
            fields = line.split(",")
            if fields[0] in claims_of_interest:
                output.write(line)

FileNotFoundError: [Errno 2] No such file or directory: 'Claim_Medical_Header.csv'

In [28]:
data_from_claims_header = pd.read_csv('caesarian_claims.csv', header=None, index_col=[0])

#### By sorting for rows based on whether they're in the previous data frame or not, the column names were lost. The following code restores those column names manually.

In [29]:
data_from_claims_header.columns = [
 'Claim_Type_Cd',
 'Claim_Type_Desc',
 'Admit_Type_Cd',
 'Admit_Type_Desc',
 'Admit_Source_Cd',
 'Admit_Source_Desc',
 'Discharge_Status_Cd',
 'Discharge_Status_Desc',
 'Bill_Type_Cd',
 'Third_Party_Liability_Cd',
 'Third_Party_Liability_Desc',
 'E_Cd',
 'Service_Start_Dt',
 'Charge_Amt',
 'Plan_Paid_Amt',
 'Prepaid_Amt',
 'Copay_Amt',
 'Coinsurance_Amt',
 'Line_of_Business_Cd',
 'Capitation_Flag',
 'Admission_Dx_ID',
 'Principal_Dx_ID',
 'Primary_Proc_ID',
 'Member_Age_Years',
 'Length_of_Stay',
 'ER_Flag',
 'Line_Count',
 'Deductible_Amt',
 'Dental_Flag',
 'Service_Start_Year',
 'Small_Area_Cd',
 'Gender_Cd',
 'Member_Gender_Desc']

data_from_claims_header

,Claim_Type_Cd,Claim_Type_Desc,Admit_Type_Cd,Admit_Type_Desc,Admit_Source_Cd,Admit_Source_Desc,Discharge_Status_Cd,Discharge_Status_Desc,Bill_Type_Cd,Third_Party_Liability_Cd,...,Member_Age_Years,Length_of_Stay,ER_Flag,Line_Count,Deductible_Amt,Dental_Flag,Service_Start_Year,Small_Area_Cd,Gender_Cd,Member_Gender_Desc
0,,,,,,,,,,,,,,,,,,,,,
171290596,P,Professional,9,INFORMATION NOT AVAILABLE,9,Information Not Available,0,Unknown,UK,1,...,45,0.0,N,1,0.00,N,2016,56.0,F,Female
238759510,P,Professional,9,INFORMATION NOT AVAILABLE,9,Information Not Available,0,Unknown,UK,1,...,21,0.0,N,1,0.00,N,2016,16.0,F,Female
280709565,P,Professional,9,INFORMATION NOT AVAILABLE,9,Information Not Available,0,Unknown,UK,1,...,32,0.0,N,1,0.00,N,2016,17.0,F,Female
282874855,P,Professional,9,INFORMATION NOT AVAILABLE,9,Information Not Available,0,Unknown,UK,1,...,29,0.0,N,1,0.00,N,2016,41.0,F,Female
280377363,P,Professional,9,INFORMATION NOT AVAILABLE,9,Information Not Available,0,Unknown,UK,1,...,41,0.0,N,2,0.00,N,2016,22.0,F,Female
282207917,P,Professional,9,INFORMATION NOT AVAILABLE,9,Information Not Available,0,Unknown,UK,2,...,22,0.0,N,1,0.00,N,2016,12.0,F,Female
281750562,P,Professional,9,INFORMATION NOT AVAILABLE,9,Information Not Available,0,Unknown,UK,1,...,30,0.0,N,1,0.00,N,2016,30.1,F,Female
282307730,P,Professional,9,INFORMATION NOT AVAILABLE,9,Information Not Available,0,Unknown,UK,1,...,37,0.0,N,1,0.00,N,2016,41.0,F,Female
249268529,P,Professional,9,INFORMATION NOT AVAILABLE,9,Information Not Available,0,Unknown,UK,1,...,39,0.0,N,1,0.00,N,2016,51.0,F,Female


#### Many of these columns are duplicates of columns already in the dataframe, and many have single values/missing information that limits their usefulness. The two columns of potential interest to are  Member_Age_Years (age of the patient)  and Small_Area_Cd (which is the taxonomy for where the procedure took place). Isolate these columns into their own data frame, then 

In [30]:
additional_columns = data_from_claims_header.filter(['Member_Age_Years','Small_Area_Cd'], axis=1)
additional_columns

,Member_Age_Years,Small_Area_Cd
0,,
171290596,45,56.0
238759510,21,16.0
280709565,32,17.0
282874855,29,41.0
280377363,41,22.0
282207917,22,12.0
281750562,30,30.1
282307730,37,41.0
249268529,39,51.0


#### To add the additional columns onto the data frame of interest, perform a join as follows:

In [34]:
additional_header_df = simplified_CPT4_df.join(additional_columns)
additional_header_df

,Place_of_Service_Cd,CPT4_ID,Service_Start_Dt,Charge_Amt,Prepaid_Amt,Plan_Paid_Amt,Capitation_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt,Member_Age_Years,Small_Area_Cd
139773905,21,59510,2016-12-13,4005.00,0.0,1219.06,N,0.0,0.00,1828.60,25,38.0
139905888,21,59510,2016-04-20,3850.00,0.0,2602.60,N,0.0,354.90,0.00,30,36.0
139934599,21,59510,2016-08-02,3200.00,0.0,1617.52,N,0.0,429.98,0.00,38,2.0
139935657,21,59510,2016-11-07,3800.00,0.0,1382.85,N,0.0,0.00,892.15,30,39.0
139962718,21,59510,2016-12-05,3600.00,0.0,2138.50,N,0.0,136.50,0.00,34,33.2
139987927,21,59510,2016-07-17,4743.00,0.0,1478.02,N,0.0,104.93,1500.00,20,54.0
140066615,11,59510,2016-03-18,3850.00,0.0,2957.50,N,0.0,0.00,0.00,30,18.0
140095735,21,59510,2016-12-23,6308.00,0.0,1619.64,N,0.0,143.91,846.41,32,23.1
140121674,21,59510,2016-09-16,4005.00,0.0,1881.94,N,0.0,165.72,1000.00,23,48.0
140148389,21,59510,2016-05-24,6308.00,0.0,724.50,N,0.0,136.50,0.00,21,25.0


## Feature Engineering 

#### Feature Engineering is the process of creating the features necessary for machine learning algorithms to work. It relies on a combination of insight from domain knowledge and finagling to transform the raw data into indicators that improve the predictive accuracy of the model on test data the model hasn't seen yet. 

#### A To Do List to get the data frame ready for modeling

* Categorize 'Date_of_Service' as a weekend or not; Create a binary 
* Add columns to turn the following into numeric binaries (algorithms work best with numeric data):
    * Capitation Flag
    * Coinsurance
    * Prepaid
    * Deductible
    * CoPay
    * Hospital v. Non-Hospital
* Additional get_dummies for location (big_city or not; mid_sized_city or not; rural_or_not)

In [ ]:
#create binaries 
additional_header_df['Prepaid_or_not'] = (additional_header_df['Prepaid_Amt'] > 0).astype(int)
#Repeat for other columns
additional_header_df['Deductible_or_Not'] = (additional_header_df['Deductible_Amt'] >0).astype(int)
additional_header_df['Coinsurance_Amt'] = (additional_header_df['Coinsurance_Amt'] >0).astype(int)
additional_header_df['Copay_or_not'] = (additional_header_df['Copay_Amt'] >0).astype(int)
additional_header_df['Capitation_Flag_Binary'] = (additional_header_df['Capitation_Flag'] == 'Y').astype(int)
#Take a look to make sure the columns have been created as intended
additional_header_df.head()

In [46]:
#Differentiate between weekends and non-weekends- make a binary column for weekends
is_saturday = pd.to_datetime(additional_header_df['Service_Start_Dt']).dt.dayofweek == 5
is_sunday = pd.to_datetime(additional_header_df['Service_Start_Dt']).dt.dayofweek == 6
is_weekend = is_saturday | is_sunday

In [47]:
is_weekend.astype(int)
additional_header_df['is_weekend'] = (is_weekend).astype(int)
additional_header_df.head()

,Place_of_Service_Cd,CPT4_ID,Service_Start_Dt,Charge_Amt,Prepaid_Amt,Plan_Paid_Amt,Capitation_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt,Member_Age_Years,Small_Area_Cd,inpatient_hospital,small_city,large_city,rural,is_weekend
139773905,21,59510,2016-12-13,4005.0,0.0,1219.06,N,0.0,0.00,1828.60,25,38.0,0,1,0,0,0
139905888,21,59510,2016-04-20,3850.0,0.0,2602.60,N,0.0,354.90,0.00,30,36.0,0,1,0,0,0
139934599,21,59510,2016-08-02,3200.0,0.0,1617.52,N,0.0,429.98,0.00,38,2.0,0,0,0,1,0
139935657,21,59510,2016-11-07,3800.0,0.0,1382.85,N,0.0,0.00,892.15,30,39.0,0,1,0,0,0
139962718,21,59510,2016-12-05,3600.0,0.0,2138.50,N,0.0,136.50,0.00,34,33.2,0,0,1,0,0


#### Overwhelmingly the procedure was performed at an Inpatient Hospital. Because of this, it makes good sense to categorize the setting of the procedure  as hospital or not hospital

##### Categorical Definitions for Place_of_Service_Cd

* 11 - Office 
* 12 - Home 
* 21 - Inpatient Hospital 
* 22 - On-campus Outpatient 
* 23 - ER 
* 99 - other 


In [48]:
additional_header_df['inpatient_hospital'] = np.where(additional_header_df['Place_of_Service_Cd']==1, '1', '0').astype(int)
additional_header_df

,Place_of_Service_Cd,CPT4_ID,Service_Start_Dt,Charge_Amt,Prepaid_Amt,Plan_Paid_Amt,Capitation_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt,Member_Age_Years,Small_Area_Cd,inpatient_hospital,small_city,large_city,rural,is_weekend
139773905,21,59510,2016-12-13,4005.00,0.0,1219.06,N,0.0,0.00,1828.60,25,38.0,0,1,0,0,0
139905888,21,59510,2016-04-20,3850.00,0.0,2602.60,N,0.0,354.90,0.00,30,36.0,0,1,0,0,0
139934599,21,59510,2016-08-02,3200.00,0.0,1617.52,N,0.0,429.98,0.00,38,2.0,0,0,0,1,0
139935657,21,59510,2016-11-07,3800.00,0.0,1382.85,N,0.0,0.00,892.15,30,39.0,0,1,0,0,0
139962718,21,59510,2016-12-05,3600.00,0.0,2138.50,N,0.0,136.50,0.00,34,33.2,0,0,1,0,0
139987927,21,59510,2016-07-17,4743.00,0.0,1478.02,N,0.0,104.93,1500.00,20,54.0,0,0,0,1,1
140066615,11,59510,2016-03-18,3850.00,0.0,2957.50,N,0.0,0.00,0.00,30,18.0,0,0,1,0,0
140095735,21,59510,2016-12-23,6308.00,0.0,1619.64,N,0.0,143.91,846.41,32,23.1,0,0,1,0,0
140121674,21,59510,2016-09-16,4005.00,0.0,1881.94,N,0.0,165.72,1000.00,23,48.0,0,1,1,0,0
140148389,21,59510,2016-05-24,6308.00,0.0,724.50,N,0.0,136.50,0.00,21,25.0,0,0,1,0,0


### Handling the Geographic Data
#### There's too many codes in the Small_Area_Cd taxonomy to create a get_dummies column for all of them. Instead, to simplify handling the data, it makes sense to split them into three groups based off of population from the 2010 census in each area. These splits were:
* Big City (Population >100,000): 15, 17, 18, 19, 21, 24, 25, 22, 23.1, 47, 48, 49, 33.2, 34.1, 34.2, 35
* Small City (Population < 100,000; referenced as a city not a county): 1, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16, 20, 26, 27, 28, 28, 29.1, 30.1, 31, 32, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 48, 58, 60
* Rural (only referenced as a county): 2, 4, 6, 40, 50, 51, 52, 53, 54, 55, 56, 57, 61

In [49]:
# Create a function to turn location information into a binary for each of the splits
def print_col_value(x):
    small_city_val = [1, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16, 20, 
                  26, 27, 28, 28, 29.1, 30.1, 31, 32, 36, 37, 38, 
                  39, 41, 42, 43, 44, 45, 46, 48, 58, 60]
    
    if x in small_city_val:
        return 1
    else:
        return 0
  

In [50]:
def large_col_value(x):
    big_city = [15, 17, 18, 19, 21, 24, 25, 22, 23.1, 47, 48, 49, 33.2, 34.1, 34.2, 35]
    
    if x in big_city:
        return 1
    else:
        return 0

In [51]:
def rural_col_value(x):
    rural_val = [2, 4, 6, 40, 50, 51, 52, 53, 54, 55, 56, 57, 61]
    
    if x in rural_val:
        return 1
    else:
        return 0

In [52]:
additional_header_df['small_city'] = additional_header_df['Small_Area_Cd'].apply(lambda x: print_col_value(x))

In [53]:
additional_header_df['large_city'] = additional_header_df['Small_Area_Cd'].apply(lambda x: large_col_value(x))

In [54]:
additional_header_df['rural'] = additional_header_df['Small_Area_Cd'].apply(lambda x: rural_col_value(x))

In [55]:
additional_header_df.head()

,Place_of_Service_Cd,CPT4_ID,Service_Start_Dt,Charge_Amt,Prepaid_Amt,Plan_Paid_Amt,Capitation_Flag,Copay_Amt,Coinsurance_Amt,Deductible_Amt,Member_Age_Years,Small_Area_Cd,inpatient_hospital,small_city,large_city,rural,is_weekend
139773905,21,59510,2016-12-13,4005.0,0.0,1219.06,N,0.0,0.00,1828.60,25,38.0,0,1,0,0,0
139905888,21,59510,2016-04-20,3850.0,0.0,2602.60,N,0.0,354.90,0.00,30,36.0,0,1,0,0,0
139934599,21,59510,2016-08-02,3200.0,0.0,1617.52,N,0.0,429.98,0.00,38,2.0,0,0,0,1,0
139935657,21,59510,2016-11-07,3800.0,0.0,1382.85,N,0.0,0.00,892.15,30,39.0,0,1,0,0,0
139962718,21,59510,2016-12-05,3600.0,0.0,2138.50,N,0.0,136.50,0.00,34,33.2,0,0,1,0,0


### Data is ready for modeling!